# L7 Process Inputs: Chaining Prompts

- 복잡한 작업을 하나의 프롬프트로 해결하지 않고 여러개의 프롬프트로 분리해서 작업

- 첫번째 프롬프트의 결과를 두번째 프롬프트의 입력으로 넣는 식으로 진행하기 때문에 Chain이란 표현을 씀

- 하나의 프롬프트에서 CoT를 사용해서 비슷하게 할 수 있는데 왜 분할할까?
    - gpt-4 이상의 고급 모델은 분할하지 않아도 잘하는 경향이 있음
    - 하지만 분할하면
        - 원리적으로 LLM의 부담을 줄일 수 있음(프롬프트 관리의 쉬움과 오류 가능성 감소), 마치 코딩할 때 작은 작업 단위로 함수를 만드는 것과 같은 개념으로 보면 됨
        - 예를 들면 고객 응대 챗봇은 제일 먼저 고객 문의의 카테고리를 분류하고 분류된 카테고리에 따라서 이후 프롬프트가 챗봇에서 응답을 생성하게 하는 것이 효율적
        - 기술적으로는 짧은 프롬프트를 사용하여 비용을 절감하는 효과를 누릴 수 있음
        - 분할된 단계의 종류에 따른 실패율 같은것을 분석해서 프롬프트를 더 정교하게 만들 수 있고 필요하다면 (현재까지는 필요할 수 있음) 프롬프트와 프롬프트 사이에 사람이 개입해서 프로그램을 진행시킬 수 있음
        - 마지막으로 고급 개념인 외부 Tools를 이용할 수 있게 할 수 있음


## Setup

In [1]:
# 구글 코립일 경우
!pip install -qU openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 6.0 MB/s eta 0:00:00


In [2]:
import os
import openai

from openai import OpenAI
from google.colab import userdata

client = OpenAI(
    api_key=userdata.get('OPENAI_API_KEY')
)


#### helper function


In [3]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens, # 모델이 출력하는 최대 토큰
    )

    return response.choices[0].message.content

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )

    return response.choices[0].message.content


## 복잡한 작업을 여러 프롬프트로 구현하기



### 단계 1: 관련 제품 및 카테고리 이름 추출

In [49]:
delimiter = "####"

# 고객이 문의 한 제품의 카테고리와 해당 제품을
# {
#     category: 해당 카테고리
#     product: 해당 제품
# }
# 형식으로 출력하고
# 특히 제품 리스트에서 해당하는 것이 없으면 빈리스트로 출력할 것

system_message = f"""
고객 서비스 문의가 {delimiter} 문자로 구분되어 제공됩니다.

다음과 같이 파이썬 사전 형식을 출력합니다:
    'category': <Computers and Laptops,
                 Smartphones and Accessories,
                 Televisions and Home Theater Systems,
                 Gaming Consoles and Accessories,
                 Audio Equipment,
                 Cameras and Camcorders 중 하나>,
    'products': <아래 [허용되는 제품 목록]에 있는 제품의 리스트>

반드시 지켜야 할 사항
1. category와 product의 쌍을 별도의 사전으로 리스트에 담아 출력합니다.
2. 출력에 포함되는 카테고리와 제품은 고객 서비스 문의에서 반드시 언급된 것이어야 합니다.
3. 제품이 언급된 경우 아래 허용되는 제품 목록에서 올바른 카테고리와 연결되어 있어야 합니다.
4. 카테고리만 언급된 경우 products 키 없이 category 키만 포함 합니다.
5. 제품이나 카테고리를 찾을 수 없으면 빈 목록을 출력합니다.

허용되는 제품 목록:
Computers and Laptops category:
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

Smartphones and Accessories category:
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

Televisions and Home Theater Systems category:
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

Gaming Consoles and Accessories category:
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

Audio Equipment category:
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

Cameras and Camcorders category:
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

다른 항목 없이 오직 상품과 카테고리 목록만 출력합니다.
"""


In [52]:
user_message_1 = f"""
smartx pro phone 하고 fotosnap camera에 대해서 알려주세요. DSLR이요.
그리고 tv에 대해서 더 좀 알려주세요."""

messages =  [
    {'role':'system', 'content': system_message},
    {'role':'user', 'content': f"{delimiter}{user_message_1}{delimiter}"},
]

category_and_product_response_1 = get_completion_from_messages(messages, model='gpt-4o-mini')

print(category_and_product_response_1)
print(type(category_and_product_response_1))

[
    {'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']},
    {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
    {'category': 'Televisions and Home Theater Systems'}
]
<class 'str'>


In [53]:
user_message_2 = f"""내 공유기가 고장난것 같아요."""

messages =  [
    {'role':'system', 'content': system_message},
    {'role':'user', 'content': f"{delimiter}{user_message_2}{delimiter}"},
]

# 공유기가 없어서 빈 리스트를 출력하는 것이 의도된 응답
# 제품 목록은 영어로 되어 있고 공유기란 제품 카테고리는 한국어로 질의되었지만
# gpt-4경우 목록에 공유기가 없다는 것을 잘 판단함
response = get_completion_from_messages(messages, model='gpt-4o-mini')
print(response)
print(type(response))

[]
<class 'str'>


### 단계 2: 카테고리와 제품의 데이터 베이스



In [54]:
# 위에서 언급된 제품에 대한 스펙 데이터 베이스
products = {
    "TechPro Ultrabook": {
        "name": "TechPro Ultrabook",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-UB100",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["13.3-inch display", "8GB RAM", "256GB SSD", "Intel Core i5 processor"],
        "description": "A sleek and lightweight ultrabook for everyday use.",
        "price": 799.99
    },
    "BlueWave Gaming Laptop": {
        "name": "BlueWave Gaming Laptop",
        "category": "Computers and Laptops",
        "brand": "BlueWave",
        "model_number": "BW-GL200",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["15.6-inch display", "16GB RAM", "512GB SSD", "NVIDIA GeForce RTX 3060"],
        "description": "A high-performance gaming laptop for an immersive experience.",
        "price": 1199.99
    },
    "PowerLite Convertible": {
        "name": "PowerLite Convertible",
        "category": "Computers and Laptops",
        "brand": "PowerLite",
        "model_number": "PL-CV300",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["14-inch touchscreen", "8GB RAM", "256GB SSD", "360-degree hinge"],
        "description": "A versatile convertible laptop with a responsive touchscreen.",
        "price": 699.99
    },
    "TechPro Desktop": {
        "name": "TechPro Desktop",
        "category": "Computers and Laptops",
        "brand": "TechPro",
        "model_number": "TP-DT500",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["Intel Core i7 processor", "16GB RAM", "1TB HDD", "NVIDIA GeForce GTX 1660"],
        "description": "A powerful desktop computer for work and play.",
        "price": 999.99
    },
    "BlueWave Chromebook": {
        "name": "BlueWave Chromebook",
        "category": "Computers and Laptops",
        "brand": "BlueWave",
        "model_number": "BW-CB100",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["11.6-inch display", "4GB RAM", "32GB eMMC", "Chrome OS"],
        "description": "A compact and affordable Chromebook for everyday tasks.",
        "price": 249.99
    },
    "SmartX ProPhone": {
        "name": "SmartX ProPhone",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-PP10",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["6.1-inch display", "128GB storage", "12MP dual camera", "5G"],
        "description": "A powerful smartphone with advanced camera features.",
        "price": 899.99
    },
    "MobiTech PowerCase": {
        "name": "MobiTech PowerCase",
        "category": "Smartphones and Accessories",
        "brand": "MobiTech",
        "model_number": "MT-PC20",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["5000mAh battery", "Wireless charging", "Compatible with SmartX ProPhone"],
        "description": "A protective case with built-in battery for extended usage.",
        "price": 59.99
    },
    "SmartX MiniPhone": {
        "name": "SmartX MiniPhone",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-MP5",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["4.7-inch display", "64GB storage", "8MP camera", "4G"],
        "description": "A compact and affordable smartphone for basic tasks.",
        "price": 399.99
    },
    "MobiTech Wireless Charger": {
        "name": "MobiTech Wireless Charger",
        "category": "Smartphones and Accessories",
        "brand": "MobiTech",
        "model_number": "MT-WC10",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["10W fast charging", "Qi-compatible", "LED indicator", "Compact design"],
        "description": "A convenient wireless charger for a clutter-free workspace.",
        "price": 29.99
    },
    "SmartX EarBuds": {
        "name": "SmartX EarBuds",
        "category": "Smartphones and Accessories",
        "brand": "SmartX",
        "model_number": "SX-EB20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "24-hour battery life"],
        "description": "Experience true wireless freedom with these comfortable earbuds.",
        "price": 99.99
    },

    "CineView 4K TV": {
        "name": "CineView 4K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-4K55",
        "warranty": "2 years",
        "rating": 4.8,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "A stunning 4K TV with vibrant colors and smart features.",
        "price": 599.99
    },
    "SoundMax Home Theater": {
        "name": "SoundMax Home Theater",
        "category": "Televisions and Home Theater Systems",
        "brand": "SoundMax",
        "model_number": "SM-HT100",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["5.1 channel", "1000W output", "Wireless subwoofer", "Bluetooth"],
        "description": "A powerful home theater system for an immersive audio experience.",
        "price": 399.99
    },
    "CineView 8K TV": {
        "name": "CineView 8K TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-8K65",
        "warranty": "2 years",
        "rating": 4.9,
        "features": ["65-inch display", "8K resolution", "HDR", "Smart TV"],
        "description": "Experience the future of television with this stunning 8K TV.",
        "price": 2999.99
    },
    "SoundMax Soundbar": {
        "name": "SoundMax Soundbar",
        "category": "Televisions and Home Theater Systems",
        "brand": "SoundMax",
        "model_number": "SM-SB50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.1 channel", "300W output", "Wireless subwoofer", "Bluetooth"],
        "description": "Upgrade your TV's audio with this sleek and powerful soundbar.",
        "price": 199.99
    },
    "CineView OLED TV": {
        "name": "CineView OLED TV",
        "category": "Televisions and Home Theater Systems",
        "brand": "CineView",
        "model_number": "CV-OLED55",
        "warranty": "2 years",
        "rating": 4.7,
        "features": ["55-inch display", "4K resolution", "HDR", "Smart TV"],
        "description": "Experience true blacks and vibrant colors with this OLED TV.",
        "price": 1499.99
    },

    "GameSphere X": {
        "name": "GameSphere X",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-X",
        "warranty": "1 year",
        "rating": 4.9,
        "features": ["4K gaming", "1TB storage", "Backward compatibility", "Online multiplayer"],
        "description": "A next-generation gaming console for the ultimate gaming experience.",
        "price": 499.99
    },
    "ProGamer Controller": {
        "name": "ProGamer Controller",
        "category": "Gaming Consoles and Accessories",
        "brand": "ProGamer",
        "model_number": "PG-C100",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["Ergonomic design", "Customizable buttons", "Wireless", "Rechargeable battery"],
        "description": "A high-quality gaming controller for precision and comfort.",
        "price": 59.99
    },
    "GameSphere Y": {
        "name": "GameSphere Y",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-Y",
        "warranty": "1 year",
        "rating": 4.8,
        "features": ["4K gaming", "500GB storage", "Backward compatibility", "Online multiplayer"],
        "description": "A compact gaming console with powerful performance.",
        "price": 399.99
    },
    "ProGamer Racing Wheel": {
        "name": "ProGamer Racing Wheel",
        "category": "Gaming Consoles and Accessories",
        "brand": "ProGamer",
        "model_number": "PG-RW200",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Force feedback", "Adjustable pedals", "Paddle shifters", "Compatible with GameSphere X"],
        "description": "Enhance your racing games with this realistic racing wheel.",
        "price": 249.99
    },
    "GameSphere VR Headset": {
        "name": "GameSphere VR Headset",
        "category": "Gaming Consoles and Accessories",
        "brand": "GameSphere",
        "model_number": "GS-VR",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Immersive VR experience", "Built-in headphones", "Adjustable headband", "Compatible with GameSphere X"],
        "description": "Step into the world of virtual reality with this comfortable VR headset.",
        "price": 299.99
    },

    "AudioPhonic Noise-Canceling Headphones": {
        "name": "AudioPhonic Noise-Canceling Headphones",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-NC100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["Active noise-canceling", "Bluetooth", "20-hour battery life", "Comfortable fit"],
        "description": "Experience immersive sound with these noise-canceling headphones.",
        "price": 199.99
    },
    "WaveSound Bluetooth Speaker": {
        "name": "WaveSound Bluetooth Speaker",
        "category": "Audio Equipment",
        "brand": "WaveSound",
        "model_number": "WS-BS50",
        "warranty": "1 year",
        "rating": 4.5,
        "features": ["Portable", "10-hour battery life", "Water-resistant", "Built-in microphone"],
        "description": "A compact and versatile Bluetooth speaker for music on the go.",
        "price": 49.99
    },
    "AudioPhonic True Wireless Earbuds": {
        "name": "AudioPhonic True Wireless Earbuds",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-TW20",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["True wireless", "Bluetooth 5.0", "Touch controls", "18-hour battery life"],
        "description": "Enjoy music without wires with these comfortable true wireless earbuds.",
        "price": 79.99
    },
    "WaveSound Soundbar": {
        "name": "WaveSound Soundbar",
        "category": "Audio Equipment",
        "brand": "WaveSound",
        "model_number": "WS-SB40",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["2.0 channel", "80W output", "Bluetooth", "Wall-mountable"],
        "description": "Upgrade your TV's audio with this slim and powerful soundbar.",
        "price": 99.99
    },
    "AudioPhonic Turntable": {
        "name": "AudioPhonic Turntable",
        "category": "Audio Equipment",
        "brand": "AudioPhonic",
        "model_number": "AP-TT10",
        "warranty": "1 year",
        "rating": 4.2,
        "features": ["3-speed", "Built-in speakers", "Bluetooth", "USB recording"],
        "description": "Rediscover your vinyl collection with this modern turntable.",
        "price": 149.99
    },

    "FotoSnap DSLR Camera": {
        "name": "FotoSnap DSLR Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-DSLR200",
        "warranty": "1 year",
        "rating": 4.7,
        "features": ["24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses"],
        "description": "Capture stunning photos and videos with this versatile DSLR camera.",
        "price": 599.99
    },
    "ActionCam 4K": {
        "name": "ActionCam 4K",
        "category": "Cameras and Camcorders",
        "brand": "ActionCam",
        "model_number": "AC-4K",
        "warranty": "1 year",
        "rating": 4.4,
        "features": ["4K video", "Waterproof", "Image stabilization", "Wi-Fi"],
        "description": "Record your adventures with this rugged and compact 4K action camera.",
        "price": 299.99
    },
    "FotoSnap Mirrorless Camera": {
        "name": "FotoSnap Mirrorless Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-ML100",
        "warranty": "1 year",
        "rating": 4.6,
        "features": ["20.1MP sensor", "4K video", "3-inch touchscreen", "Interchangeable lenses"],
        "description": "A compact and lightweight mirrorless camera with advanced features.",
        "price": 799.99
    },
    "ZoomMaster Camcorder": {
        "name": "ZoomMaster Camcorder",
        "category": "Cameras and Camcorders",
        "brand": "ZoomMaster",
        "model_number": "ZM-CM50",
        "warranty": "1 year",
        "rating": 4.3,
        "features": ["1080p video", "30x optical zoom", "3-inch LCD", "Image stabilization"],
        "description": "Capture life's moments with this easy-to-use camcorder.",
        "price": 249.99
    },
    "FotoSnap Instant Camera": {
        "name": "FotoSnap Instant Camera",
        "category": "Cameras and Camcorders",
        "brand": "FotoSnap",
        "model_number": "FS-IC10",
        "warranty": "1 year",
        "rating": 4.1,
        "features": ["Instant prints", "Built-in flash", "Selfie mirror", "Battery-powered"],
        "description": "Create instant memories with this fun and portable instant camera.",
        "price": 69.99
    }
}

In [55]:
def get_product_by_name(name):
    """제품 이름으로 제품 정보를 찾아 반환"""
    return products.get(name, None)

def get_products_by_category(category):
    """카테고리로 해당 제품 정보를 모두 반환"""
    return [product for product in products.values() if product["category"] == category]

In [56]:
print(get_product_by_name("TechPro Ultrabook"))
type(get_product_by_name("TechPro Ultrabook"))

{'name': 'TechPro Ultrabook', 'category': 'Computers and Laptops', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 processor'], 'description': 'A sleek and lightweight ultrabook for everyday use.', 'price': 799.99}


dict

In [58]:
print(get_products_by_category("Computers and Laptops"))
type(get_products_by_category("Computers and Laptops")), len(get_products_by_category("Computers and Laptops"))

[{'name': 'TechPro Ultrabook', 'category': 'Computers and Laptops', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 processor'], 'description': 'A sleek and lightweight ultrabook for everyday use.', 'price': 799.99}, {'name': 'BlueWave Gaming Laptop', 'category': 'Computers and Laptops', 'brand': 'BlueWave', 'model_number': 'BW-GL200', 'warranty': '2 years', 'rating': 4.7, 'features': ['15.6-inch display', '16GB RAM', '512GB SSD', 'NVIDIA GeForce RTX 3060'], 'description': 'A high-performance gaming laptop for an immersive experience.', 'price': 1199.99}, {'name': 'PowerLite Convertible', 'category': 'Computers and Laptops', 'brand': 'PowerLite', 'model_number': 'PL-CV300', 'warranty': '1 year', 'rating': 4.3, 'features': ['14-inch touchscreen', '8GB RAM', '256GB SSD', '360-degree hinge'], 'description': 'A versatile convertible laptop with a responsive touchscreen.', 'price': 

(list, 5)

In [59]:
print(user_message_1)


smartx pro phone 하고 fotosnap camera에 대해서 알려주세요. DSLR이요.
그리고 tv에 대해서 더 좀 알려주세요.


In [60]:
print(category_and_product_response_1)
type(category_and_product_response_1)

[
    {'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']},
    {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
    {'category': 'Televisions and Home Theater Systems'}
]


str

### 단계 3: 구조화된 문자열을 파이썬 사전의 리스트로 바꾸기

- llm이 반환한 카테고리, 제품의 문자열을 파이썬 리스트, 사전으로 변환

In [61]:
import json

def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        input_string = input_string.strip("``` python json")
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None


In [62]:
category_and_product_list = read_string_to_list(category_and_product_response_1)
type(category_and_product_list), len(category_and_product_list)

(list, 3)

In [63]:
import pandas as pd

pd.DataFrame(
    {
        'content':[
            category_and_product_response_1,
            category_and_product_list
        ],
        'type':[
            type(category_and_product_response_1),
            type(category_and_product_list)
        ]
    }
)

,content,type
0,[\n {'category': 'Smartphones and Accessori...,<class 'str'>
1,"[{'category': 'Smartphones and Accessories', '...",<class 'list'>


In [64]:
# 이제 모델의 출력이 인덱싱할 수 있는 실제 파이썬 객체가 됨
print(category_and_product_list[0])
print(category_and_product_list[0]['products'])

{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}
['SmartX ProPhone']


In [65]:
# 지금까지 고객 문의와 관련된 카테고리와 제품을 뽑아낸 파이썬 리스트
# 리스트 내부의 요소는 사전으로 되어 있음
category_and_product_list

[{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']},
 {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
 {'category': 'Televisions and Home Theater Systems'}]

### 단계 4: 관련 제품 및 카테고리에 대한 자세한 제품 정보 가져오기(Retrieve)

- 단계 1, 2, 3을 거쳐 만들어진 고객이 문의한 제품의 카테고리와 제품 이름 정보를 이용해 LLM이 대답에 사용할 데이터를 만듬

- 제품의 모든 데이터 베이스를 LLM에게 제공하는 것 보다 이렇게 프롬프트의 단계를 나누고 1단계 프롬프트로 정보를 뽑아내고 이를 정리하여 2단계 프롬프트의 정보로 제공

- 입력에 포함되는 정보가 관련된 정보만으로 제한되어 오류가 없고 더 정확한 답변을 생성할 수 있음

- 입력 시퀀스가 짧아져 비용 절감 효과

- 이 개념은 향후 벡터DB에 임베딩 검색으로 연결

In [66]:
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list: # 사전을 하나씩 순회하면서
        # data : {'category':'...', 'products':[...]}
        # [
        #     {'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']},
        #     {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
        #     {'category': 'Televisions and Home Theater Systems'}
        # ]
        try:
            # 제품이 있으면 제품 정보만 뽑아오고 끝
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    # product: {제품 정보의 사전}
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            # 제품이 없고 카테고리만 있으면
            # 카테고리의 모든 제품 정보를 뽑아옴
            elif "category" in data:
                category_name = data["category"]
                # category_name:  "문자열"
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string

- `json.dumps` 함수 테스트

In [73]:
# 사전
smartx_prophone = get_product_by_name('SmartX ProPhone')
print(smartx_prophone)

type(smartx_prophone)

{'name': 'SmartX ProPhone', 'category': 'Smartphones and Accessories', 'brand': 'SmartX', 'model_number': 'SX-PP10', 'warranty': '1 year', 'rating': 4.6, 'features': ['6.1-inch display', '128GB storage', '12MP dual camera', '5G'], 'description': 'A powerful smartphone with advanced camera features.', 'price': 899.99}


dict

In [75]:
# 문자열
smartx_prophone_dump = json.dumps(smartx_prophone, indent=4)
print(smartx_prophone_dump)
type(smartx_prophone_dump)

{
    "name": "SmartX ProPhone",
    "category": "Smartphones and Accessories",
    "brand": "SmartX",
    "model_number": "SX-PP10",
    "warranty": "1 year",
    "rating": 4.6,
    "features": [
        "6.1-inch display",
        "128GB storage",
        "12MP dual camera",
        "5G"
    ],
    "description": "A powerful smartphone with advanced camera features.",
    "price": 899.99
}


str

In [76]:
product_information_for_user_message_1 = generate_output_string(category_and_product_list)

# 출력은 사용자가 문의한 내용과 관련된 제품에 대한 정보가
# 구조화된 JSON 형태의 string
print(product_information_for_user_message_1)
type(product_information_for_user_message_1)

{
    "name": "SmartX ProPhone",
    "category": "Smartphones and Accessories",
    "brand": "SmartX",
    "model_number": "SX-PP10",
    "warranty": "1 year",
    "rating": 4.6,
    "features": [
        "6.1-inch display",
        "128GB storage",
        "12MP dual camera",
        "5G"
    ],
    "description": "A powerful smartphone with advanced camera features.",
    "price": 899.99
}
{
    "name": "FotoSnap DSLR Camera",
    "category": "Cameras and Camcorders",
    "brand": "FotoSnap",
    "model_number": "FS-DSLR200",
    "warranty": "1 year",
    "rating": 4.7,
    "features": [
        "24.2MP sensor",
        "1080p video",
        "3-inch LCD",
        "Interchangeable lenses"
    ],
    "description": "Capture stunning photos and videos with this versatile DSLR camera.",
    "price": 599.99
}
{
    "name": "CineView 4K TV",
    "category": "Televisions and Home Theater Systems",
    "brand": "CineView",
    "model_number": "CV-4K55",
    "warranty": "2 years",
    "ratin

str

### 단계 5: 사용자 서비스 문의에 답변 생성하기 (제공되는 추가 정보를 이용하여)

In [89]:
system_message = f"""
당신은 전자 제품 매장의 고객 서비스 도우미입니다.
매우 간결한 답변으로 친절하고 도움이 되는 어조로 응답하세요.
사용자에 대한 응대가 종료되지 않았다면 사용자에게 관련 후속 질문을 반드시 하세요.
"""

user_message_1 = f"""
smartx pro phone 하고 fotosnap camera에 대해서 알려주세요. DSLR이요.
그리고 tv에 대해서 더 좀 알려주세요."""

messages =  [
    {'role':'system', 'content': system_message},
    {'role':'user', 'content': user_message_1},
    {
        'role':'assistant',
        'content': f"""관련 제품들의 정보:\n {product_information_for_user_message_1}"""
    },
]

final_response = get_completion_from_messages(messages, model='gpt-4o-mini')
print(final_response)

SmartX ProPhone은 6.1인치 디스플레이와 128GB 저장공간, 12MP 듀얼 카메라를 갖춘 5G 스마트폰입니다. 가격은 899.99달러입니다.

FotoSnap DSLR 카메라는 24.2MP 센서와 1080p 비디오 촬영이 가능하며, 교환 가능한 렌즈를 지원합니다. 가격은 599.99달러입니다.

CineView 4K TV는 55인치 화면에 4K 해상도와 HDR 기능을 갖춘 스마트 TV로, 가격은 599.99달러입니다. 

더 궁금한 점이 있으신가요? 어떤 제품에 대해 더 알고 싶으신가요?


In [90]:
messages =  [
    {'role':'system', 'content': system_message},
    {'role':'user', 'content': user_message_1},
    {
        'role':'assistant',
        'content': f"""관련 제품들의 정보:\n {product_information_for_user_message_1}"""
    },
    {'role':'assistant', 'content': final_response},
    {'role':'user', 'content': 'TV는 하나밖에 없나요?'},
]

final_response = get_completion_from_messages(messages, model='gpt-4o-mini')
print(final_response)

아니요, 여러 종류의 TV가 있습니다! 

1. **CineView 4K TV** - 55인치, 4K 해상도, 가격 599.99달러
2. **CineView 8K TV** - 65인치, 8K 해상도, 가격 2999.99달러
3. **CineView OLED TV** - 55인치, 4K 해상도, 가격 1499.99달러

각 TV의 특징이나 가격에 대해 더 알고 싶으신가요?
